# Learning Deep Convolutional Networks for Demosaicing

> Original paper by Nai-Sheng Syu, Yu-SHeng Chen, Yung-Yu Chuang [[1]](https://arxiv.org/pdf/1802.03769.pdf)

> Homepage for the paper: [link](http://www.cmlab.csie.ntu.edu.tw/project/Deep-Demosaic/) (As of 21.4.2019 the authors have not released the original code)

### About the notebook

This notebook and repository aims to reproduce the state-of-the-art results provided in the [1]. 

**Author:** Eemeli Saari

**Email:** saari.eemeli@gmail.com

**Modified:** 21.4.2019

> **NOTE:**

---

In [22]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
from torchsummary import summary

We'll set a standard seed for reproducibility.

In [4]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

## Data

We load the data from the authors homepage and define a Pytorch dataset that reads patches from the images.

The image patches are densely extracted $33 \times 33$ patches that are padded with zeros to make up for the missing channels.

In [5]:
from dataset import download_data, ImagePatchDataset

In [6]:
%time dataset = ImagePatchDataset(root=download_data(), sample_size=2)
len(dataset)

CPU times: user 23 ms, sys: 14.5 ms, total: 37.5 ms
Wall time: 41.8 ms


8526

In [ ]:
data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=64,
    shuffle=True
)

## DMCNN Model

We'll first define the DMCNN model described by the paper [1].

Model contains

- Feature extraction layer
  - 128 Filters with $9\times9$ kernels.
- Non-linear mapping layer
  - 64 Filters with $1\times1$ kernels.
- Reconstruction layer
  - 3 Kernels constructing the resulted colored image
  - $5\times5$ kernels.

In [7]:
class DMCNN(torch.nn.Module):
    """DMCNN
    
    Model adopted from [link](https://arxiv.org/pdf/1802.03769.pdf)
    """
    def __init__(self):
        super(DMCNN, self).__init__()
        
        self.feature_layer = torch.nn.Sequential(
            torch.nn.Conv2d(3, 128, kernel_size=9),
            torch.nn.ReLU()
        )
        
        self.mapping_layer = torch.nn.Sequential(
            torch.nn.Conv2d(128, 64, kernel_size=1),
            torch.nn.ReLU()
        )
        
        self.reconstruction_layer = torch.nn.Sequential(
            torch.nn.Conv2d(64, 3, kernel_size=5),
            torch.nn.ReLU()
        )
    
    def forward(self, x):
        out = self.feature_layer(x)
        out = self.mapping_layer(out)
        out = self.reconstruction_layer(out)
        return out


In [8]:
%time model = DMCNN()

CPU times: user 1.66 ms, sys: 2.7 ms, total: 4.36 ms
Wall time: 3.67 ms


In [9]:
summary(model, input_size=(3, 33, 33))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 25, 25]          31,232
              ReLU-2          [-1, 128, 25, 25]               0
            Conv2d-3           [-1, 64, 25, 25]           8,256
              ReLU-4           [-1, 64, 25, 25]               0
            Conv2d-5            [-1, 3, 21, 21]           4,803
              ReLU-6            [-1, 3, 21, 21]               0
Total params: 44,291
Trainable params: 44,291
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.85
Params size (MB): 0.17
Estimated Total Size (MB): 2.03
----------------------------------------------------------------


In [10]:
model = model.double()

As we can see it's a fairly shallow convolution neural network that should be fairly fast to train.

## Training the model

The Pytorch framework doesn't offer the similar sklearn type API that's supported in the Keras so we'll need to manually create the training loop.

First we'll define the the loss function as the standard $L_2$ norm.

$$ L(\theta) = \frac{1}{n} \sum_{n}^{n=1}||F(Y_i ; \theta) - X_i||^2 $$

In [ ]:
criterion = torch.nn.MSELoss()

According to the original paper the DMCNN is trained using the stochastic gradient descent with learning rate 1 for the first two layers and 0.1 for the last layer.

This comes easy to define in Pytorch as following:

In [ ]:
optimizer = torch.optim.SGD(
    params = [
        {"params": model.feature_layer.parameters(), "lr": 1},
        {"params": model.mapping_layer.parameters(), "lr": 1},
        {"params": model.reconstruction_layer.parameters(), "lr": 0.1}
    ]
)

In [ ]:
n_epochs = 15

We'll define the training loop to include the clipping to keep the gradients within -1 and 1.

In [23]:
total_step = len(data_loader)
loss_list = []
for epoch in range(n_epochs):
    epoch_loss = []
    for idx, (cfa, target) in enumerate(data_loader):
        outputs = model(cfa)
        loss = criterion(outputs, target)
        loss_list.append(loss.item())
        epoch_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()

        if idx % 10 == 0:
            #print(outputs)
            print(f'Epoch [{epoch}/{n_epochs}], Step [{idx}/{total_step}], Loss: {loss.item()}')
    epoch_stats = np.array(epoch_loss)
    print(f'\nFinished Epoch {epoch}, Loss --- mean: {epoch_stats.mean()}, std {epoch_stats.std()}\n')
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,8))
    ax1.imshow(np.array(outputs[-1].tolist()).reshape((21, 21, 3)))
    ax2.imshow(np.array(cfa[-1].tolist()).reshape((33, 33, 3)))
    ax3.imshow(np.array(target[-1].tolist()).reshape((21, 21, 3)))
    plt.show()

NameError: name 'data_loader' is not defined